In [6]:
import pandas as pd
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm 
from urllib.parse import urljoin


## Initial data

In [123]:
db_pomoci = pd.read_csv("db_pomoci.csv")
black_list = ['http://www.dc-brno.cz']
df = db_pomoci.rename(columns={"Webová stránka" : "web", "Název":"nazev"})
df = df[(~df["web"].isna()) & (~df['web'].isin(black_list))]
df.loc[df['web'].str.startswith('www'), 'web'] = df['web'].str.replace('^www', 'https://www', regex=True)
# df['web'] = df['web'].str.replace('https://www', 'www')
# df['web'] = df['web'].str.replace('www', 'https://www')
df_agg = df.groupby("web")["Telefon"].agg(list)
df_agg["email"] = df.groupby("web")["E-mail"].agg(list)
df_agg = df_agg.reset_index().rename(columns={'web': 'web', 'Telefon': 'Telefon'})
df_agg = pd.DataFrame(df_agg)

# Scraped data

db_scraped = pd.read_csv("data/29_05/df_scraped_v2.csv")
df_res = db_scraped[~db_scraped["Phone Numbers"].isna()]
df_res = df_res.rename(columns={'web': 'web', 'Telefon': 'Telefon'})#.drop("Unnamed: 0", axis= 1)
df_res = df_res.rename(columns= {"Website":"web"})
df_res = df_res.rename(columns= {"Phone Numbers":"phones_scraped"})
df_res = df_res.rename(columns= {"Emails":"emails_scraped"})


maps_df = pd.read_csv("data/maps_results.csv",sep=";")
maps_df = maps_df.rename(columns={'Web': 'web', 'Phone': 'Telefon Mapy'})
mapy = maps_df[["web","Telefon Mapy"]]#,"Telefon Mapy"


In [124]:
print("df_agg", df_agg.shape)
print("df_res", df_res.shape)
print("df_mapy", mapy.shape)



df_agg (871, 2)
df_res (599, 3)
df_mapy (1456, 2)


In [125]:
df_res['phones_scraped'] = df_res['phones_scraped'].str.split(', ')
mapy['Telefon Mapy'] = mapy['Telefon Mapy'].str.split(', ')
df_res = df_res[["web","phones_scraped"]]
df_res['source'] = 'scraped'
df_agg['source'] = 'db'
mapy['source'] = 'mapy.cz'
#df_res['emails_scraped'] = df_res['emails_scraped'].str.split(', ')
#df_agg = df_agg.rename(columns= {"Telefon":"phones_scraped"})
df_res_exp = df_res.explode('phones_scraped').reset_index(drop=True)
mapy_exp = mapy.explode('Telefon Mapy').reset_index(drop=True)
mapy_exp = mapy_exp.rename(columns= {"Telefon Mapy":"Telefon"})
df_res_exp = df_res.rename(columns= {"phones_scraped":"Telefon"})
df_agg_exp = df_agg.explode('Telefon').reset_index(drop=True)
df_union = pd.concat([df_agg_exp, df_res_exp,mapy_exp])

/var/folders/rd/l77bltt96_97jdz4bbkmq5z40000gn/T/ipykernel_46796/1908990668.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapy['Telefon Mapy'] = mapy['Telefon Mapy'].str.split(', ')
/var/folders/rd/l77bltt96_97jdz4bbkmq5z40000gn/T/ipykernel_46796/1908990668.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapy['source'] = 'mapy.cz'


In [126]:
mapy_exp

,web,Telefon,source
0,http://www.pdz.cz,778 526 216,mapy.cz
1,http://www.cdzopava.cz/,553 652 433,mapy.cz
2,http://www.cdzopava.cz/,723 457 998,mapy.cz
3,http://www.pdz.cz,469 626 383,mapy.cz
4,http://www.pdz.cz,774 915 755,mapy.cz
...,...,...,...
1634,www.praha.ymca.cz,775 317 814,mapy.cz
1635,http://www.ymkarium.ymca.cz,737 629 149,mapy.cz
1636,www.dixie.ymca.cz,737 872 116,mapy.cz
1637,http://www.usti.ymca.cz,475 220 831,mapy.cz


In [127]:
df_union.groupby("source").count()

,web,Telefon
source,,
db,1986,1971
mapy.cz,1267,1609
scraped,599,599


In [132]:
df_union[df_union["web"]=="www.zvole.info"]

,web,Telefon,source
1533,www.zvole.info,NaN,mapy.cz


In [128]:
set_unmatched = set(df_union[df_union["Telefon"].isna() & df_union["web"]!= 'email' ]["web"])


In [129]:
len(set_unmatched)

1009

In [63]:
#df_union['Telefon'] = df_union['Telefon'].str.replace(r'\s+', '', regex=True)
#df_grouped = pd.DataFrame(df_union.groupby('web').count().sort_values(by="Telefon",ascending=False))

In [130]:
df_grouped = df_union.groupby('web').count()

# Filter the DataFrame to get rows where the count of 'Telefon' is less than 1
filtered_df = df_grouped[df_grouped['Telefon'] < 1]

# Extract the list of 'web' values
web_list = filtered_df.index.tolist()
print(len(web_list))
for c in web_list:
    print(c)

6
http://www.ckp-dobrichovice.cz
http://www.fokus-mb.cz/klubko
https://www.milicak.cz
misa.zapletalova@tiscali.cz
www.milicak.cz
www.zvole.info


In [134]:
#df_union[df_union["web"] == 'asistovanekontakty.cz'].sort_values(by="Telefon")
web = 'https://www.zvole.info'
df_agg[df_agg["web"] == web].sort_values(by="Telefon")
df_res[df_res["web"] == web].sort_values(by="phones_scraped")

,web,phones_scraped,source
761,https://www.zvole.info,"[+420 257 760 337, 257 760 337]",scraped


In [14]:
preview = df_res.merge(df_agg , on="web")
preview = preview.merge(mapy_tel , on="web")
preview['phones_scraped'] = preview['phones_scraped'].str.split(', ')
preview


,web,phones_scraped,Telefon,Telefon Mapy
0,http://www.pdz.cz,"[121 407 9329, 64 24 22 18, 420 466 400 116]","[778 526 216, 469 626 383, 774 915 755]",778 526 216
1,http://www.pdz.cz,"[121 407 9329, 64 24 22 18, 420 466 400 116]","[778 526 216, 469 626 383, 774 915 755]","469 626 383, 774 915 755"
2,http://www.medsix.cz/?option=com_content&view=...,"[483 341 232, 483 300 087, 483 310 919, 483 34...",[483 341 267],483 341 267
3,https://jindrichuvhradec.charita.cz/sluzba/852/,"[731 402 982, 731 604 554, 605 849 877]","[731 402 982, 605 849 877]","731 402 982, 605 849 877"
4,https://www.homeopatiecz.com/,"[777 169 278, 736 660 315, 777 146 760, 725 76...",[736 660 315],736 660 315
...,...,...,...,...
612,https://www.firmy.cz/detail/13179143-phdr-tere...,[420 376 322 688],[376 322 688],376 322 688
613,https://search.seznam.cz/?q=psychiatrick%C3%A1...,[420608971030],[736 132 143],736 132 143
614,http://ssp-ol.cz/,[585 427 141],[774 406 453],774 406 453
615,http://www.psychoterapeuti.cz/adresar-psychote...,"[284 016 451, 736 660 315, 530 334 600, 775 18...",[466 611 601],466 611 601


In [1]:
df_agg.head()

NameError: name 'df_agg' is not defined

In [135]:
v1 = pd.read_csv('data/df_scraped.csv')
v2 = pd.read_csv('data/29_05/df_scraped_v2.csv')

In [138]:
v1.head()
#v2.head()

,Unnamed: 0,Website,Phone Numbers
0,0.0,http://www.pdz.cz,"121 407 9329, 64 24 22 18, 420 466 400 116"
1,1.0,http://www.medsix.cz/?option=com_content&view=...,"483 341 232, 483 300 087, 483 310 919, 483 341..."
2,2.0,https://www.osbonanza.cz,NaN
3,3.0,https://www.svetlokadanzs.cz,NaN
4,4.0,https://jindrichuvhradec.charita.cz/sluzba/852/,"731 402 982, 731 604 554, 605 849 877"


In [143]:
v1.isna().sum()

Unnamed: 0         2
Website            2
Phone Numbers    310
dtype: int64

In [140]:
v2.isna().sum()

Website            0
Emails           364
Phone Numbers    267
dtype: int64

In [141]:
v1.shape

(872, 3)

In [142]:
v2.shape

(866, 3)

In [1]:
import pandas as pd
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re



phone_regex = re.compile(r"""
    (?:
        \+420[-\s]?        # Optional country code +420 followed by an optional space or dash
    )?
    (?:
        \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
        \d{3}[-\s]?        # Second part of the phone number (3 digits) followed by an optional space or dash
        \d{3}              # Third part of the phone number (3 digits)
        |                  # OR
        \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
        \d{2}[-\s]?        # Second part of the phone number (2 digits) followed by an optional space or dash
        \d{2}[-\s]?        # Third part of the phone number (2 digits) followed by an optional space or dash
        \d{2}              # Fourth part of the phone number (2 digits)
    )
""", re.VERBOSE)
email_regex = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")

# Test the regex
test_numbers = [
    "+420 123 456 789",
    "+420-123-456-789",
    "123 456 789",
    "123 456 789 123",
    "123456789123",
    "774 97 97 14",
    "+420 774 97 97 14",
    "+420 565 552 604"
]

for number in test_numbers:
    match = phone_regex.search(number)
    if match:
        print(f"Matched: {match.group()}")
    else:
        print(f"No match: {number}")

Matched: +420 123 456 789
Matched: +420-123-456-789
Matched: 123 456 789
Matched: 123 456 789
Matched: 123456789
Matched: 774 97 97 14
Matched: +420 774 97 97 14
Matched: +420 565 552 604


In [2]:
df_scraped = pd.read_csv("../data/1_06/df_scraped_full.csv")

In [3]:
df_scraped.shape

(1882, 6)

In [4]:
df_scraped.head()

,Unnamed: 0,Base Website,Scraped Page,Page Type,Emails,Phone Numbers
0,0,http://www.adelafararova.cz/,http://www.adelafararova.cz/,main,255akleda77@gmail.comPal,"613 3827 66, 776 871 255, 670100-221, +420 776..."
1,1,http://www.adelafararova.cz/,http://www.adelafararova.cz/#kontakt,contact,255akleda77@gmail.comPal,"776 871 255, 670100-221, +420 776 871 255"
2,2,http://www.adelafararova.cz/,http://www.adelafararova.cz/template/contact,contact,255akleda77@gmail.comPal,"613 3827 66, +420 776 871 255"
3,3,http://www.ckp-dobrichovice.cz,http://www.ckp-dobrichovice.cz,main,"seminare@ckp-dobrichovice.cz, info@ckp-dobrich...","+420 777 773 014, +420 775 603 573, +420 773 3..."
4,4,http://www.ckp-dobrichovice.cz,https://www.ckp-dobrichovice.cz/kontakt/,contact,info@ckp-dobrichovice.cz,"+420 773 339 916, +420 723 603 600"


In [5]:
df_scraped[df_scraped["Phone Numbers"].isna()]

,Unnamed: 0,Base Website,Scraped Page,Page Type,Emails,Phone Numbers
14,14,http://www.medsix.cz,http://www.medsix.cz/web/jpol.html,contact,NaN,NaN
16,16,http://www.cheiront.cz,http://www.cheiront.cz/o-nas,contact,NaN,NaN
19,19,http://www.cheiront.cz/,http://www.cheiront.cz/o-nas,contact,NaN,NaN
23,23,http://obcanskaporadna.cz,http://obcanskaporadna.cz/o-nas/,contact,NaN,NaN
32,32,http://www.clovekvtisni.cz,https://klima.clovekvtisni.cz,contact,barbora.vrablikova@clovekvtisni.czKontakty,NaN
...,...,...,...,...,...,...
1832,181,https://www.psychoterapie-vysocina.cz,https://www.psychoterapie-vysocina.cz,main,NaN,NaN
1833,182,https://www.psycholog-jesenik.cz/,https://www.psycholog-jesenik.cz/,main,NaN,NaN
1836,185,https://www.zachranny-kruh.cz/,https://www.zachranny-kruh.cz/o-nas/,contact,NaN,NaN
1841,190,https://www.psychocentrumdomecek.cz,https://www.psychocentrumdomecek.cz/o-nas,contact,NaN,NaN


In [35]:
df_res = pd.read_csv("../data/1_06/df_scraped_full.csv")
df_res['phones_scraped'] = df_res['Phone Numbers'].str.split(', ')
df_res['emails_scraped'] = df_res['Emails'].str.split(', ')
df_res_phones = df_res[["Base Website", "Scraped Page","phones_scraped"]]
print(df_res_phones.shape)
df_res_exp = df_res_phones.explode('phones_scraped').reset_index(drop=True)
print(df_res_exp.shape)
# mapy['Telefon Mapy'] = mapy['Telefon Mapy'].str.split(', ')
# df_res = df_res[["web","phones_scraped"]]
# df_res['source'] = 'scraped'
# df_agg['source'] = 'db'
# mapy['source'] = 'mapy.cz'
# #df_agg = df_agg.rename(columns= {"Telefon":"phones_scraped"})
# df_res_exp = df_res.explode('phones_scraped').reset_index(drop=True)
# mapy_exp = mapy.explode('Telefon Mapy').reset_index(drop=True)
# mapy_exp = mapy_exp.rename(columns= {"Telefon Mapy":"Telefon"})
# df_res_exp = df_res.rename(columns= {"phones_scraped":"Telefon"})
# df_agg_exp = df_agg.explode('Telefon').reset_index(drop=True)
# df_union = pd.concat([df_agg_exp, df_res_exp,mapy_exp])

(1882, 3)
(16134, 3)


In [44]:
import phonenumbers
import phonenumbers.geocoder
import pandas
def py_parse_phonenumber(num):
    try:
        parsed_num = phonenumbers.parse(num, 'CZ')
        phonenumbers.is_possible_number_with_reason(parsed_num)
        return {
            'formated_number':phonenumbers.format_number(parsed_num, phonenumbers.PhoneNumberFormat.E164),
            'number': parsed_num.national_number,
            'prefix': parsed_num.country_code,
            'country_code': phonenumbers.region_code_for_number(parsed_num),
            'valid': phonenumbers.is_valid_number(parsed_num),
            'possible': phonenumbers.is_possible_number(parsed_num),
            'parsed': True
        }
    except Exception as e:
        return {'number': num, 'prefix': None, 'country_code': None, 'valid': False, 'possible': False, 'parsed': False}

def udf(df: pd.DataFrame, column_name: str):
    results = df[column_name].apply(py_parse_phonenumber)
    parsed_df = pd.DataFrame(results.tolist())
    return parsed_df

def explode_df(df:pd.DataFrame, column_name:str) -> pd.DataFrame:
    #df_res = pd.read_csv("../data/1_06/df_scraped_full.csv")
    df_res = df
    df_res[f'{column_name}_scraped'] = df_res[column_name].str.split(', ')
   # df_res['emails_scraped'] = df_res['Emails'].str.split(', ')
    df_res_phones = df_res[["Base Website", "Scraped Page",f"{column_name}_scraped"]]
    df_res_exp = df_res_phones.explode(f'{column_name}_scraped').reset_index(drop=True)
    return df_res_exp

ress_df = udf(df_res_exp,"phones_scraped")

In [39]:
df_res_1 = pd.read_csv("../data/1_06/df_scraped_1_06.csv")
df_res_2 = pd.read_csv("../data/1_06/df_scraped_2_1_06.csv")
df_scraped_1_06 = pd.concat([df_res_1,df_res_2])
df_scraped_1_06.to_csv("../data/1_06/df_scraped_1_06_full.csv")

# Phones

In [103]:
def has_more_than_5_consecutive_zeros(number):
    return bool(re.search(r'0{4,}', str(number)))

# Apply the function and filter out rows


df_res_1 = pd.read_csv("../data/1_06/df_scraped_1_06_full.csv")

phones_exp = explode_df(df_res_1,"Phone Numbers")
ress_df = udf(phones_exp,"Phone Numbers_scraped")
phones_exp["formated_number"] = ress_df["formated_number"]
phones_exp.drop(columns=["Phone Numbers_scraped"], inplace= True)
phones_deduped = phones_exp.drop_duplicates(subset=['Base Website', 'formated_number'])
filtered_df = phones_deduped[~phones_deduped['formated_number'].apply(has_more_than_5_consecutive_zeros)]
phones_df = filtered_df.sort_values(by="formated_number", ascending=True)
phones_df[phones_df["Base Website"] == "http://www.ppp.opava.cz/"]
# ress_df.head()


,Base Website,Scraped Page,formated_number
1390,http://www.ppp.opava.cz/,http://www.ppp.opava.cz/,+420239455230
1408,http://www.ppp.opava.cz/,http://www.ppp.opava.cz/,+420250268268
1324,http://www.ppp.opava.cz/,http://www.ppp.opava.cz/,+420250306252
1291,http://www.ppp.opava.cz/,http://www.ppp.opava.cz/,+420255480246
1017,http://www.ppp.opava.cz/,http://www.ppp.opava.cz/,+420258276276
1100,http://www.ppp.opava.cz/,http://www.ppp.opava.cz/,+420258306267
1501,http://www.ppp.opava.cz/,http://www.ppp.opava.cz/,+420319520420
963,http://www.ppp.opava.cz/,http://www.ppp.opava.cz/,+420331547423
1325,http://www.ppp.opava.cz/,http://www.ppp.opava.cz/,+420355399347
1416,http://www.ppp.opava.cz/,http://www.ppp.opava.cz/,+420386507507


In [ ]:
df_res_1_exp.head(50)

# Emails

In [91]:
def clean_email(email):
    if pd.isna(email):
        return email
    # Regular expression to find valid email addresses
    email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b')
    # Find all valid email addresses in the string
    valid_emails = email_pattern.findall(email)
    # Fix emails that have additional symbols after ".cz"
    cleaned_emails = []
    for email in valid_emails:
        if email.endswith('.cz'):
            cleaned_emails.append(email)
        else:
            em1 = re.sub(r'\.cz.*', '.cz', email)            
            em2 = re.sub(r'\.com.*', '.com', em1)            
            em3 = re.sub(r'\.eu.*', '.eu', em2)            
            cleaned_emails.append(re.sub(r'\.org.*', '.org', em3))
    return ', '.join(cleaned_emails)

df_res_1 = pd.read_csv("../data/1_06/df_scraped_1_06_full.csv")

df_res_1_exp = explode_df(df_res_1,"Emails")
df_res_1_exp.head()

# Apply the clean_email function to the "Emails" column
df_res_1_exp['Emails_scraped'] = df_res_1_exp['Emails_scraped'].apply(clean_email)

df_deduped = df_res_1_exp.drop_duplicates(subset=['Base Website', 'Emails_scraped'])
filtered_df = df_deduped[
    (~df_deduped["Emails_scraped"].isna()) #& 
    #(~df_deduped["Emails_scraped"].str.endswith('.cz', na=False))
]

# Sort the filtered DataFrame
sorted_df = filtered_df.sort_values(by="Emails_scraped", ascending=False)
sorted_df

# Display or save the sorted DataFrame
# df_res_1_exp.drop_duplicates(subset=['Base Website', 'Emails_scraped'])[~(df_res_1_exp["Emails_scraped"].isna()) & ~(df_res_1_exp["Emails_scraped"].str.endswith('.cz'))].sort_values(by="Emails_scraped",ascending=False)

,Base Website,Scraped Page,Emails_scraped
185,http://www.leccos.cz,http://www.leccos.cz,zvonice@leccos.cz
6955,https://www.frydekmistek.cz,https://www.frydekmistek.cz,zvakova.eva@frydekmistek.cz
6985,https://www.frydekmistek.cz,https://www.frydekmistek.cz,zuzanakova.lucie@frydekmistek.cz
3095,http://www.sasmcb.cz,http://www.sasmcb.cz,zuzana.vodakova@sasmcb.cz
4572,http://www.tadyated.org/,http://www.tadyated.org/,zuzana.vitovcova@tadyated.org
...,...,...,...
5600,https://www.prostor-plus.cz,https://www.prostor-plus.cz,
209,http://www.fokuslabe.cz,http://www.fokuslabe.cz,
2061,http://www.fokus-mb.cz,http://www.fokus-mb.cz,
4642,http://www.vidacr.cz/,http://www.vidacr.cz/,


In [92]:
sorted_df.to_csv('../data/2_06/emails.csv')